In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Hi,

* This is pretty simple solution which achieved 0.25230 on LB.
* I used random forests but Gradientboosting can be used with little modifications.
* There are 2 predefined hyperparameters settings which you can use for both RF and Gradient Boosting.

In [ ]:
!pip3 install tensorflow_decision_forests

In [ ]:
import tensorflow_decision_forests as tfdf
import tensorflow as tf
from tensorflow import keras

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
# I didn't include date time column and also used 3 models for each target variable
X_train_model_1 = train.drop(['date_time','target_benzene','target_nitrogen_oxides'],axis = 1)
X_train_model_2 = train.drop(['date_time','target_carbon_monoxide','target_nitrogen_oxides'],axis = 1)
X_train_model_3 = train.drop(['date_time','target_carbon_monoxide','target_benzene'],axis = 1)

In [ ]:
columns = test.columns[1:]
X_test = test[columns]
columns

In [ ]:
# better_default and benchmark_rank1 are two predefined hyperparameters settings for both RF and Gradient Boost.
tfdf.keras.RandomForestModel.predefined_hyperparameters()

In [ ]:
# Name of the label column.
label = "target_carbon_monoxide"

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(X_train_model_1, label=label, task=tfdf.keras.Task.REGRESSION)
model_1 = tfdf.keras.RandomForestModel(hyperparameter_template="benchmark_rank1", task=tfdf.keras.Task.REGRESSION)
model_1.fit(train_ds)

In [ ]:
# summary is very useful for feature importance and other statictics of model
# model_1.summary()

In [ ]:
# converting test dataframe to keras dataset
test = tfdf.keras.pd_dataframe_to_tf_dataset(X_test, task=tfdf.keras.Task.REGRESSION)

In [ ]:
submission['target_carbon_monoxide'] = model_1.predict(test)

In [ ]:
# Name of the label column.
label = "target_benzene"

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(X_train_model_2, label=label, task=tfdf.keras.Task.REGRESSION)
model_2 = tfdf.keras.RandomForestModel(hyperparameter_template="benchmark_rank1", task=tfdf.keras.Task.REGRESSION)
model_2.fit(train_ds)

In [ ]:
submission['target_benzene'] = model_2.predict(test)

In [ ]:
# Name of the label column.
label = "target_nitrogen_oxides"

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(X_train_model_3, label=label, task=tfdf.keras.Task.REGRESSION)
model_3 = tfdf.keras.RandomForestModel(hyperparameter_template="benchmark_rank1", task=tfdf.keras.Task.REGRESSION)
model_3.fit(train_ds)

In [ ]:
submission['target_nitrogen_oxides'] = model_3.predict(test)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission_rf_tf.csv', index=False)